# RCB'S BEST SEASON WAS 2016, THEIR SQUAD WAS 

**https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-squad-969877/series-squads**

In [1]:
stads = [
	"IND: I.S. Bindra Punjab Cricket Association Stadium",
	"IND: Sawai Mansingh Stadium, Jaipur",
	"IND: Eden Gardens, Kolkata",
	"IND: Wankhede Stadium, Mumbai",
	"IND: M.Chinnaswamy Stadium, Bengaluru",
	"IND: Narendra Modi Stadium, Motera, Ahmedabad",
	"IND: Arun Jaitley Stadium, Delhi",
	"IND: Rajiv Gandhi International Stadium, Uppal, Hyderabad",
	"IND: Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",
	"IND: MA Chidambaram Stadium, Chepauk, Chennai"
]
homes = [
    "PunjabKings",
    "RajasthanRoyals",
    "KolkataKnightRiders",
    "MumbaiIndians",
    "RoyalChallengersBangalore",
    "GujaratTitans",
    "DelhiCapitals",
    "SunrisersHyderabad",
    "LucknowSupergiants",
    "ChennaiSuperKings"
]

                                                      template

In [2]:
import re
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
options = Options()
options.headless = True
import time

In [3]:
data = [
    {
        #allrounder
        "role": None,
        "strike rate": None,
        "wickets": None,
        "average": None,
        "awards": None,
        "sixes": None,
        "fours": None,
        "economy": None,
        "batting average": None,
        "bowling average": None,
        "bat_style": None,
        "bowl_style": None,
        "importance": None,
    },
    {
        #bowler
        "role": None,
        "wickets": None,
        "stadium1 economy": None,
        "stadium10 economy": None,
        "awards": None,
        "economy": None,
        "bowling average": None,
        "bat_style": None,
        "bowl_style": None,
        "importance": None,
    },
    {
        #batsman
        "role": None,
        "strike rate": None,
        "runs": None,
        "stadium1 average": None,
        "stadium10 strike rate": None,
        "bat_style": None,
        "bowl_style": None,
        "awards": None,
        "sixes": None,
        "fours": None,
        "economy": None,
        "batting average": None,
        "importance": None,
    }
]

In [4]:
class Player:
    def __init__(self,name,age,role):
        self.name = name
        self.age = age   
        self.role = role
squad = {}

finding rosters and cleaning them up

In [ ]:
urls = ['https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-squad-969877/series-squads']
for url in urls:
    rqsts = requests.get(url)
    soup = BeautifulSoup(rqsts.content,'lxml')
    names = soup.find_all('div', class_ = 'ds-flex ds-flex-row ds-items-center ds-justify-between')
    names = [name.text.strip() for name in names]
    names = [name.replace('(c)', '').replace('†', '') for name in names if "Withdrawn" not in name]
    ages = soup.find_all('span', class_ = 'ds-text-compact-xxs ds-font-bold')
    ages = [age.text.strip() for age in ages]
    ages = [int(re.search(r'(\d+)y', age).group(1)) for age in ages if re.search(r'(\d+)y', age)]
    roles = soup.find_all('p',class_ = 'ds-text-tight-s ds-font-regular ds-mb-2 ds-mt-1')
    roles = [role.text.strip() for role in roles]
    for i in range(len(roles)):
        if 'Batter' in roles[i]:
            roles[i] = 'Batter'
        elif 'Allrounder' in roles[i]:
            roles[i] = 'AllRounder'
        elif 'Bowler' in roles[i]:
            roles[i] = 'Bowler'
    removes = len(names) - len(ages)
    n = removes
    del ages[n:]
    del roles[n:]
    for name, age, role in zip(names, ages, roles):
        print(name, age, role)
        squad[name] = Player(name=name,age=age,role=role)

making a function that takes stuff from statsguru

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

def stats_taking(player, i):
    driver = webdriver.Chrome(options=options)
    driver.get('https://stats.espncricinfo.com/ci/engine/stats/index.html')
    driver.maximize_window()
    search_box = driver.find_element(By.NAME, "search")
    search_query = player.name
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    link = driver.find_element(By.XPATH, "//a[starts-with(text(), 'Players and Officials')]")
    link.click()
    link = driver.find_element(By.XPATH, "//a[text()='Twenty20 matches player']")
    link.click()
    time.sleep(3)
    menu_url = driver.current_url
    player_info = driver.find_element(By.XPATH, "//p[@style='padding-bottom:10px']").text
    details = player_info.split(' - ')[1].strip()
    styles = details.split('; ') 
    bat_style = styles[0]
    data[i]['bat_style'] = bat_style
    print(bat_style)
    if len(styles) > 1:
        bowl_style = styles[1]
        data[i]['bowl_style'] = bowl_style
        print(bowl_style)
    try:
        field_style = styles[2]
        data[i]['field_style'] = field_style
        print(field_style)
    except:
        pass
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='cumulative']")
    radio_button.click()
    time.sleep(0.1)
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='awards_match']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    
    try:
        player_of_match_elements = driver.find_elements(By.XPATH, "//td[text()='player of the match']")
        potm_awards = len(player_of_match_elements)
        data[i]['awards'] = int(potm_awards)
        print(f'potm awards {potm_awards}')
        driver.get(menu_url)
    except Exception as e:
        driver.get(menu_url)
    
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='results']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    time.sleep(0.1)
    
    try:
        wins = driver.find_elements(By.XPATH, "//td[text()='won']")
        losses = driver.find_elements(By.XPATH, "//td[text()='lost']")
        importance = len(wins) / len(losses)
        data[i]['importance'] = importance
        driver.get(menu_url)
    except Exception as e:
        pass
    
    if player.role == 'AllRounder':
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(3)
        
        row = driver.find_element(By.XPATH, "//tr[@class='data1']")
        cells = row.find_elements(By.TAG_NAME, "td")
        economy = float(cells[9].text)
        wickets = int(cells[6].text)
        bowling_average = float(cells[8].text)
        data[i]['wickets'] = wickets
        data[i]['bowling average'] = bowling_average
        data[i]['economy'] = economy
        driver.get(menu_url)
        
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(3)
        
        table_row = driver.find_element(By.CSS_SELECTOR, "tr.data1")
        cells = table_row.find_elements(By.TAG_NAME, "td")
        runs = int(cells[4].text)
        data[i]['runs'] = runs
        batting_average = float(cells[7].text)
        data[i]['batting_average'] = batting_average
        strike_rate = float(cells[9].text)
        data[i]['strike_rate'] = strike_rate
        fours = int(cells[12].text)
        data[i]['fours'] = fours
        sixes = int(cells[13].text)
        data[i]['sixes'] = sixes
    elif player.role == 'Bowler':
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(3)
        
        row = driver.find_element(By.XPATH, "//tr[@class='data1']")
        cells = row.find_elements(By.TAG_NAME, "td")
        economy = float(cells[9].text)
        wickets = int(cells[6].text)
        bowling_average = float(cells[8].text)
        data[i]['wickets'] = wickets
        data[i]['bowling average'] = bowling_average
        data[i]['economy'] = economy
        driver.get(menu_url)
        
        for stadium, home in zip(stads, homes):
            dropdown = driver.find_element(By.NAME, "ground")
            select = Select(dropdown)
            select.select_by_visible_text(stadium)
            radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")
            radio_button.click()
            driver.get(menu_url)
            filtered_rows = driver.find_elements(By.XPATH, '//tr[td[@class="left" and text()="filtered"]]')
            for row in filtered_rows:
                specific_column = row.find_element(By.XPATH, './td[10]')
                break
            specific_column = specific_column.text
            data[i][f'{home} economy'] = float(specific_column)
            driver.get(menu_url)
    elif player.role == 'Batter':
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(3)
        
        table_row = driver.find_element(By.CSS_SELECTOR, "tr.data1")
        cells = table_row.find_elements(By.TAG_NAME, "td")
        runs = int(cells[5].text)
        data[i]['runs'] = runs
        batting_average = float(cells[7].text)
        data[i]['batting_average'] = batting_average
        strike_rate = float(cells[9].text)
        data[i]['strike_rate'] = strike_rate
        fours = int(cells[13].text)
        data[i]['fours'] = fours
        sixes = int(cells[14].text)
        data[i]['sixes'] = sixes
        print(f"Runs: {runs}, Batting Average: {batting_average}, Strike Rate: {strike_rate}, Fours: {fours}, Sixes: {sixes}")
        driver.get(menu_url)
        for stadium, home in zip(stads, homes):
            dropdown = driver.find_element(By.NAME, "ground")
            select = Select(dropdown)
            try:
                select.select_by_visible_text(stadium)
                radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
                radio_button.click()
                driver.get(menu_url)
                filtered_rows = driver.find_elements(By.XPATH, '//tr[td[@class="left" and text()="filtered"]]')
                for row in filtered_rows:
                    specific_column = row.find_element(By.XPATH, './td[5]')
                    specific_column = specific_column.text
                    data[i][f'{home} average'] = float(specific_column)
                    print(f"{home} average: {specific_column}")
                    break
            except:
                print(f"{home} average: N/A")
                continue
            driver.get(menu_url)
    driver.quit()

In [ ]:
for i,player in enumerate(squad.values()):
    stats_taking(player, i)

writing into a json file

In [11]:
# Write to a JSON file
with open("rcb_dataset.json", "w") as f:
    json.dump(data, f, indent=4)  # `indent=4` makes it human-readable